In [2]:
import pandas as pd
import numpy as np
import sklearn
import scipy.stats as ss
import matplotlib.pyplot as plt

# Homework 2

## 1.3

In [15]:
edge_cols = ['Node1_id', 'Node2_id', 'Edge_weight', 'Edge_type', 'Source_id', 'Line_num']
edges = pd.read_csv('data/9606.hn_HS_CX.edge', sep='\t', names=edge_cols)

In [16]:
edges

,Node1_id,Node2_id,Edge_weight,Edge_type,Source_id,Line_num
0,ENSG00000284589,ENSG00000276821,0.000008,hn_HS_CX,humannet.HumanNet,248612
1,ENSG00000284589,ENSG00000267534,0.000006,hn_HS_CX,humannet.HumanNet,332085
2,ENSG00000284589,ENSG00000178802,0.000006,hn_HS_CX,humannet.HumanNet,318214
3,ENSG00000284589,ENSG00000172772,0.000007,hn_HS_CX,humannet.HumanNet,312029
4,ENSG00000284589,ENSG00000167751,0.000008,hn_HS_CX,humannet.HumanNet,243030
...,...,...,...,...,...,...
154382,ENSG00000000419,ENSG00000124802,0.000005,hn_HS_CX,humannet.HumanNet,394893
154383,ENSG00000000419,ENSG00000101557,0.000006,hn_HS_CX,humannet.HumanNet,360110
154384,ENSG00000000419,ENSG00000101224,0.000006,hn_HS_CX,humannet.HumanNet,328424
154385,ENSG00000000419,ENSG00000096401,0.000006,hn_HS_CX,humannet.HumanNet,321013


In [10]:
node_map = pd.read_csv('data/9606.hn_HS_CX.node_map', sep='\t', header=None)

In [11]:
node_map

,0,1,2,3,4
0,ENSG00000187555,ENSG00000187555,Gene,USP7,ubiquitin specific peptidase 7 [Source:HGNC Sy...
1,ENSG00000196431,ENSG00000196431,Gene,CRYBA4,crystallin beta A4 [Source:HGNC Symbol;Acc:HGN...
2,ENSG00000166889,ENSG00000166889,Gene,PATL1,"PAT1 homolog 1, processing body mRNA decay fac..."
3,ENSG00000119042,ENSG00000119042,Gene,SATB2,SATB homeobox 2 [Source:HGNC Symbol;Acc:HGNC:2...
4,ENSG00000183508,ENSG00000183508,Gene,FAM46C,family with sequence similarity 46 member C [S...
...,...,...,...,...,...
10933,ENSG00000116903,ENSG00000116903,Gene,EXOC8,exocyst complex component 8 [Source:HGNC Symbo...
10934,ENSG00000047346,ENSG00000047346,Gene,FAM214A,family with sequence similarity 214 member A [...
10935,ENSG00000134901,ENSG00000134901,Gene,KDELC1,KDEL motif containing 1 [Source:HGNC Symbol;Ac...
10936,ENSG00000160180,ENSG00000160180,Gene,TFF3,trefoil factor 3 [Source:HGNC Symbol;Acc:HGNC:...
